In [1]:
# Install necessary packages, including sentencepiece for LlamaTokenizer
%pip install transformers
%pip install sentencepiece
%pip install SpeechRecognition
%pip install pygame
%pip install playsound
%pip install pyaudio
%pip install pyttsx3
%pip install torch


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
# Step 1: Import necessary libraries
from transformers import LlamaTokenizer, LlamaForCausalLM
import torch 

# Step 2: Use a publicly available lighter LLaMA model to prevent kernel crashes
model_name = "meta-llama/Llama-2-7b-chat"  # Lighter LLaMA model

# Step 3: Load the tokenizer and model from Hugging Face, without legacy issues
tokenizer = LlamaTokenizer.from_pretrained(model_name, legacy=False)  # Set legacy=False to avoid warning
model = LlamaForCausalLM.from_pretrained(model_name)

# Step 4: Define function to generate response using the LLaMA model
def generate_response(user_input):
    inputs = tokenizer(user_input, return_tensors="pt", truncation=True, padding=True, max_length=150, padding_token=eos_token)
    attention_mask = inputs['attention_mask']
    
    # Generate response with attention mask
    output = model.generate(inputs.input_ids, attention_mask=attention_mask, max_length=150)
    bot_response = tokenizer.decode(output[0], skip_special_tokens=True)
    
    return bot_response


KeyboardInterrupt: 

In [ ]:
# Step 3: Speech Recognition, Text-to-Speech, and Audio Output Setup

import os
import pygame
from gtts import gTTS
import tempfile
import speech_recognition as sr
from time import sleep
import datetime

# Function to capture speech input from microphone
def capture_speech(mic_index):
    recognizer = sr.Recognizer()
    recognizer.energy_threshold = 300  # Lower energy threshold for faster detection
    mic = sr.Microphone(device_index=mic_index)
    
    with mic as source:
        print("Listening for your query...")
        recognizer.adjust_for_ambient_noise(source, duration=0.5)  # Optimized ambient noise detection
        
        try:
            audio = recognizer.listen(source, timeout=5, phrase_time_limit=6)  # Slightly reduced timeout

        except sr.WaitTimeoutError:
            print("Listening timed out, please speak again.")
            return None  # Return None if timeout occurs

    try:
        user_input = recognizer.recognize_google(audio)
        print(f"User said: {user_input}")
        return user_input
    except sr.UnknownValueError:
        print("Sorry, I could not understand the audio.")
        return None
    except sr.RequestError as e:
        print(f"Error with the Google Speech Recognition service; {e}")
        return None

# Function to synthesize speech and play the response
def speak_response(text):
    tts = gTTS(text=text, lang='en')
    with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as temp_audio_file:
        tts.save(temp_audio_file.name)
        temp_file_path = temp_audio_file.name
    
    pygame.mixer.init()
    try:
        pygame.mixer.music.load(temp_file_path)
        pygame.mixer.music.play()

        # Check music play in smaller intervals for reduced waiting time
        while pygame.mixer.music.get_busy():
            pygame.time.wait(50)

    except Exception as e:
        print(f"Error playing audio: {e}")
    finally:
        pygame.mixer.quit()
        if os.path.exists(temp_file_path):
            os.remove(temp_file_path)


In [ ]:
# Step 4: Handle Time and Date Queries, and Select the Microphone Automatically

# Function to handle date and time queries
def handle_time_date_queries(user_input):
    if "time" in user_input.lower():
        current_time = datetime.datetime.now().strftime("%I:%M %p")
        return f"The current time is {current_time}."
    elif "date" in user_input.lower() or "today" in user_input.lower():
        current_date = datetime.datetime.now().strftime("%A, %B %d, %Y")
        return f"Today is {current_date}."
    else:
        return None

# Function to automatically select the microphone (e.g., index 2)
def select_microphone():
    mic_list = sr.Microphone.list_microphone_names()
    index = 2  # Automatically select microphone at index 2
    return index


In [ ]:
# Step 5: Wake Word Detection and Main Conversational Loop

# Function to listen for the wake word "Hey Amigo"
def listen_for_wake_word(mic_index):
    recognizer = sr.Recognizer()
    mic = sr.Microphone(device_index=mic_index)
    
    print("Waiting for wake word 'Hey Amigo'...")
    while True:
        with mic as source:
            recognizer.adjust_for_ambient_noise(source, duration=0.5)
            audio = recognizer.listen(source, timeout=5)  # Timeout added to avoid long waits
            
        try:
            user_input = recognizer.recognize_google(audio)
            if "hey amigo" in user_input.lower():
                print("Wake word detected!")
                return True  # Wake word detected, start conversation
        except sr.UnknownValueError:
            continue
        except sr.RequestError as e:
            print(f"Error with the Google Speech Recognition service: {e}")
            continue

# Main conversation loop function
def amigo_conversational_companion():
    mic_index = select_microphone()

    # Step 1: Wait for the wake word "Hey Amigo"
    if listen_for_wake_word(mic_index):
        greeting = "Hi Hello Namaskara, I am Amigo. How can I help you today?"
        print(f"Amigo: {greeting}")
        speak_response(greeting)
        
        # Step 2: Enter conversation loop
        while True:
            user_input = capture_speech(mic_index)
            
            if user_input is None:
                continue

            # Exit if the user says "exit" or "goodbye"
            if "exit" in user_input.lower() or "goodbye" in user_input.lower():
                farewell_message = "Goodbye, have a nice day!"
                print(f"Amigo: {farewell_message}")
                speak_response(farewell_message)
                break

            # Check for date/time queries
            time_date_response = handle_time_date_queries(user_input)
            if time_date_response:
                print(f"Amigo: {time_date_response}")
                speak_response(time_date_response)
                continue
            
            # Generate a bot response using the LLaMA model
            bot_response = generate_response(user_input)
            print(f"Amigo: {bot_response}")
            speak_response(bot_response)



In [ ]:
# Run the conversational companion
if __name__ == "__main__":
    amigo_conversational_companion()
